# GBasis Tutorial - Evaluations

# Table of Contents  

- [Evaluations](#1-Evaluations)
    - [1.1 Evaluation of Contractions](#1.1-Evaluation-of-Contractions)
    - [1.2 Evaluation of Derivatives of Contractions](#1.2-Evaluation-of-Derivatives-of-Contractions)
    - [1.3 Evaluations of Density Related Properties](#1.3-Evaluations-of-Density-Related-Properties)
        - [1.3.1 Density](#1.3.1-Density)
        - [1.3.2 Arbitrary Derivatives of Density](#1.3.2-Arbitrary-Derivatives-of-Density)
        - [1.3.3 Gradient of Density](#1.3.3-Gradient-of-Density)
        - [1.3.4 Laplacian of Density](#1.3.4-Laplacian-of-Density)
        - [1.3.5 Hessian of Density](#1.3.5-Hessian-of-Density)
    - [1.4 Evaluations of Density Matrix Related Properties](#1.4-Evaluations-of-Density-Matrix-Related-Properties)
        - [1.4.1 Stress Tensor](#1.4.1-Stress-Tensor)
        - [1.4.2 Ehrenfest Force](#1.4.2-Ehrenfest-Force)
        - [1.4.3 Ehrenfest Hessian](#1.4.3-Ehrenfest-Hessian)
        - [1.4.4 Positive-Definite Kinetic Energy Density](#1.4.4-Positive-Definite-Kinetic-Energy-Density)
        - [1.4.5 General Form of Kinetic Energy Density](#1.4.5-General-Form-of-Kinetic-Energy-Density)

In the examples in this notebook, we will ubiquitously make use of global variables that are defined in the code cell below. These examples will generally use a Krypton atom as the system of study, represented with the STO-6G basis set (`basis`). The STO-6G basis set assigns 18 basis functions (atomic orbitals, AOs) to Krypton. We define an arbitrary transformation matrix (`transform`) that can be used to obtain 18 molecular orbitals (MOs) from these 18 atomic orbitals. A one-electron density matrix is generated randomly. Also, we randomly select Cartesian coordinates to represent rudimentary grids for showcasing functions in the `gbasis.evals` modules. After most code blocks, outputs of calculations, shapes of said outputs, or other clarifying information will be provided to justify that things work properly.

In [1]:
import gbasis
import numpy as np
from gbasis.parsers import parse_nwchem, make_contractions

# create an STO-6G basis in spherical coordinates for Kr (18 basis functions total)
basis_dict = parse_nwchem("data_sto6g.nwchem")
basis = make_contractions(basis_dict, ["Kr"], np.array([[0, 0, 0]]), coord_types="spherical")

# define a tranformation matrix to obtain 18 molecular orbitals from 18 atomic orbitals
transform = np.random.rand(18, 18)

# the following represents a one-electron density matrix
one_dm = np.random.rand(18, 18)
one_dm += one_dm.T

# define Cartesian coordinates of points in space (in atomic units) where basis functions are evaluated
one_point = np.random.rand(1, 3)
ten_points = np.random.rand(10, 3)

# 1 Evaluations

## 1.1 Evaluation of Contractions

The functions in the module `gbasis.evals.eval` return the evaluations of the contractions at different
coordinates:

$$\begin{equation}
  \phi(\mathbf{r}_n | \mathbf{R}_{A}, \mathbf{a}_j, \mathbf{d}_k, \boldsymbol{\alpha})
\end{equation}$$

The returned value is an array whose rows corresponds to the basis function and columns corresponds to the coordinate, $\mathbf{r}_n$.These functions can be used to find the values of the orbitals at various points, such as a grid.

In [2]:
from gbasis.evals.eval import evaluate_basis

# evalute atomic orbitals at a ten points
atomic_orb_evaluation = evaluate_basis(basis, ten_points)

# evaluate molecular orbitals at ten points
molecular_orb_evaluation = evaluate_basis(basis, ten_points, transform)

# verify shapes of evaluations
print("Shape of Atomic Orbital Evaluation (Number of AOs, Number of Evaluation Points):")
print(atomic_orb_evaluation.shape)
print()
print("Shape of Molecular Orbital Evaluation (Number of MOs, Number of Evaluation Points):")
print(molecular_orb_evaluation.shape)

Shape of Atomic Orbital Evaluation (Number of AOs, Number of Evaluation Points):
(18, 10)

Shape of Molecular Orbital Evaluation (Number of MOs, Number of Evaluation Points):
(18, 10)


## 1.2 Evaluation of Derivatives of Contractions

In `gbasis`, contractions can be derivatized to arbitrary orders. The functions in module `gbasis.evals.eval_deriv`
return the evaluations of the given derivative of the contractions at different coordinates.

$$\begin{equation}
  \frac{\partial^{m_x + m_y + m_z}}{\partial x^{m_x} \partial y^{m_y} \partial z^{m_z}}
  \phi(\mathbf{r}_n | \mathbf{R}_{A}, \mathbf{a}_j, \mathbf{d}_k, \boldsymbol{\alpha})
\end{equation}$$

The returned value is an array whose rows corresponds to the basis function and columns corresponds to the coordinate, $\mathbf{r}_n$.

In the example code cell below, presume the following derivative of the contraction is desired:

$$\frac{\partial ^{3}}{\partial x \partial y^{2}}$$

In [3]:
from gbasis.evals.eval_deriv import evaluate_deriv_basis

# evaluate the derivatives of the atomic orbitals at ten points
atomic_basis_deriv = evaluate_deriv_basis(basis, ten_points, np.array([1, 2, 0]))

# evaluate the derivatives of the molecular orbitals at ten points
molecular_basis_deriv = evaluate_deriv_basis(basis, ten_points, np.array([1, 2, 0]), transform)

# verify shapes of derivative evaluations
print("Shape of Atomic Orbital Derivative Evaluation (Number of AOs, Number of Evaluation Points):")
print(atomic_basis_deriv.shape)
print()
print("Shape of Molecular Orbital Derivative Evaluation (Number of MOs, Number of Evaluation Points):")
print(molecular_basis_deriv.shape)

Shape of Atomic Orbital Derivative Evaluation (Number of AOs, Number of Evaluation Points):
(18, 10)

Shape of Molecular Orbital Derivative Evaluation (Number of MOs, Number of Evaluation Points):
(18, 10)


## 1.3 Evaluations of Density Related Properties

The functions in the module `gbasis.evals.density` return the evaluations of the
density and its derivatives. Note that the examples below are not to be taken literally. The one-electron density matrices, expressed with respect to either atomic or molecular orbitals, were separately generated via `np.random.rand` and thus have no connection to one another. This method is not a bona fide way of approaching things.

### 1.3.1 Density

$$\rho(\mathbf{r}_n) = \sum_{ij} \gamma_{ij} \phi_i(\mathbf{r}_n) \phi_j(\mathbf{r}_n)$$

In [4]:
from gbasis.evals.density import evaluate_density

# evaluate the density at ten points using the density matrix expressed w.r.t. atomic orbitals
atomic_basis_density = evaluate_density(one_dm, basis, ten_points)

# evaluate the density at ten points using the density matrix expressed w.r.t. molecular orbitals
molecular_basis_density = evaluate_density(one_dm, basis, ten_points, transform)

print("Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(atomic_basis_density)
print()
print("Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(molecular_basis_density)

Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[ 8.33053535  1.52485541 13.66611092  0.78182131  0.94300825  5.35898483
  3.64079124 29.41080977  1.06547365  1.03792558]

Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[ 679.27207596  115.74140734 1106.38325166   58.55386393   69.60493446
  438.69154932  257.80109904 2463.42491985   78.15377575   76.98588485]


### 1.3.2 Arbitrary Derivatives of Density

$$\frac{\partial^{L_x + L_y + L_z}}{\partial x^{L_x} \partial y^{L_y} \partial z^{L_z}}
  \rho(\mathbf{r}_n)
  =
  \sum_{l_x=0}^{L_x} \sum_{l_y=0}^{L_y} \sum_{l_z=0}^{L_z}
  \binom{L_x}{l_x} \binom{L_y}{l_y} \binom{L_z}{l_z}
  \sum_{ij} \gamma_{ij}
  \frac{\partial^{l_x + l_y + l_z} \rho(\mathbf{r}_n)}{\partial x^{l_x} \partial y^{l_y} \partial z^{l_z}}
  \frac{
    \partial^{L_x + L_y + L_z - l_x - l_y - l_z} \rho(\mathbf{r}_n)
  }{
    \partial x^{L_x - l_x} \partial y^{L_y - l_y} \partial z^{L_z - l_z}
  }$$
  
In the example code cell below, presume the following derivative of the density is desired:

$$\frac{\partial ^{3}}{\partial x \partial y^{2}}$$

In [5]:
from gbasis.evals.density import evaluate_deriv_density

# evaluate the derivative of density at one point using the density matrix expressed w.r.t. atomic orbitals
atomic_basis_density_deriv = evaluate_deriv_density(np.array([1, 2, 0]), one_dm, basis, one_point)

# evaluate the derivative of density at one point using the density matrix expressed w.r.t. molecular orbitals
molecular_basis_density_deriv = evaluate_deriv_density(np.array([1, 2, 0]), 
                                                       one_dm, 
                                                       basis, 
                                                       one_point, 
                                                       transform)

print("Derivative of Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(atomic_basis_density_deriv)
print()
print("Derivative of Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(molecular_basis_density_deriv)

Derivative of Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[33.01117312]

Derivative of Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[2242.20899611]


### 1.3.3 Gradient of Density

$$
\begin{equation}
  \nabla \rho(\mathbf{r}_n)
  =
  \begin{bmatrix}
    \frac{\partial}{\partial x} \rho(\mathbf{r}_n)\\\\
    \frac{\partial}{\partial y} \rho(\mathbf{r}_n)\\\\
    \frac{\partial}{\partial z} \rho(\mathbf{r}_n)
  \end{bmatrix}\\
\end{equation}
$$

In [6]:
from gbasis.evals.density import evaluate_density_gradient

# evaluate the gradient of density at one point using the density matrix expressed w.r.t. atomic orbitals
atomic_basis_density_grad = evaluate_density_gradient(one_dm, basis, one_point)

# evaluate the gradient of density at one point using the density matrix expressed w.r.t. molecular orbitals
molecular_basis_density_grad = evaluate_density_gradient(one_dm, basis, one_point, transform)

print("Gradient of Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(atomic_basis_density_grad)
print()
print("Gradient of Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(molecular_basis_density_grad)

Gradient of Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[[ 1.71079027 -4.36098206 -3.77213831]]

Gradient of Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[[ 133.12673406 -307.9045288  -268.61004731]]


### 1.3.4 Laplacian of Density

$$
\begin{equation}
  \nabla^2 \rho(\mathbf{r}_n)
  =
  \frac{\partial^2}{\partial x^2} \rho(\mathbf{r}_n)
  + \frac{\partial^2}{\partial y^2} \rho(\mathbf{r}_n)
  + \frac{\partial^2}{\partial z^2} \rho(\mathbf{r}_n)
\end{equation}
$$

In [7]:
from gbasis.evals.density import evaluate_density_laplacian

# evaluate the Laplacian of density at one point using the density matrix expressed w.r.t. atomic orbitals
atomic_basis_density_laplacian = evaluate_density_laplacian(one_dm, basis, one_point)

# evaluate the Laplacian of density at one point using the density matrix expressed w.r.t. molecular orbitals
molecular_basis_density_laplacian = evaluate_density_laplacian(one_dm, basis, one_point, transform)

print("Laplacian of Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(atomic_basis_density_laplacian)
print()
print("Laplacian of Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(molecular_basis_density_laplacian)

Laplacian of Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[19.47865914]

Laplacian of Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[1426.08708519]


### 1.3.5 Hessian of Density

$$
\begin{equation}
  H[\rho(\mathbf{r}_n)]
  =
  \begin{bmatrix}
    \frac{\partial^2}{\partial x^2} \rho(\mathbf{r}_n) &
    \frac{\partial^2}{\partial x \partial y} \rho(\mathbf{r}_n) &
    \frac{\partial^2}{\partial x \partial z} \rho(\mathbf{r}_n)\\\\
    \frac{\partial^2}{\partial x \partial y} \rho(\mathbf{r}_n) &
    \frac{\partial^2}{\partial y^2} \rho(\mathbf{r}_n)&
    \frac{\partial^2}{\partial y \partial z} \rho(\mathbf{r}_n)\\\\
    \frac{\partial^2}{\partial x \partial z} \rho(\mathbf{r}_n) &
    \frac{\partial^2}{\partial z^2} \rho(\mathbf{r}_n)&
    \frac{\partial^2}{\partial x \partial z} \rho(\mathbf{r}_n)\\
  \end{bmatrix}\\
\end{equation}
$$

In [8]:
from gbasis.evals.density import evaluate_density_hessian

# evaluate the Hessian of density at one point using the density matrix expressed w.r.t. atomic orbitals
atomic_basis_density_hess = evaluate_density_hessian(one_dm, basis, one_point)

# evaluate the Hessian of density at one point using the density matrix expressed w.r.t. molecular orbitals
molecular_basis_density_hess = evaluate_density_hessian(one_dm, basis, one_point, transform)

print("Hessian of Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(atomic_basis_density_hess)
print()
print("Hessian of Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(molecular_basis_density_hess)

Hessian of Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[[[-7.21338935 -7.48871076 -8.31582116]
  [-7.48871076 17.21624711 23.27535782]
  [-8.31582116 23.27535782  9.47580137]]]

Hessian of Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[[[-521.07876175 -541.82466814 -614.24187133]
  [-541.82466814 1207.473431   1660.34860197]
  [-614.24187133 1660.34860197  739.69241593]]]


## 1.4 Evaluations of Density Matrix Related Properties

Given the density matrix,

$$
\begin{equation}
  \gamma(\mathbf{r}_1, \mathbf{r}_2)
  = \sum_{ij} \gamma_{ij} \phi_i(\mathbf{r}_1) \phi_j(\mathbf{r}_2)
\end{equation}
$$

many properties can be defined by evaluating the derivatives of the density
matrix at the same coordinate:

$$
\begin{equation}
  \left.
    \frac{\partial^{p_x + p_y + p_z}}{\partial x_1^{p_x} \partial y_1^{p_y} \partial z_1^{p_z}}
    \frac{\partial^{q_x + q_y + q_z}}{\partial x_2^{q_x} \partial y_2^{q_y} \partial z_2^{q_z}}
    \gamma(\mathbf{r}_1, \mathbf{r}_2)
  \right|_{\mathbf{r}_1 = \mathbf{r}_2 = \mathbf{r}_n} =
  \sum_{ij} \gamma_{ij}
  \left.
    \frac{\partial^{p_x + p_y + p_z}}{\partial x_1^{p_x} \partial y_1^{p_y} \partial z_1^{p_z}}
    \phi_i(\mathbf{r}_1)
  \right|_{\mathbf{r}_1 = \mathbf{r}_n}
  \left.
    \frac{\partial^{q_x + q_y + q_z}}{\partial x_2^{q_x} \partial y_2^{q_y} \partial z_2^{q_z}}
    \phi_j(\mathbf{r}_2)
  \right|_{\mathbf{r}_1 = \mathbf{r}_n}
\end{equation}
$$

where $\mathbf{r}_1$ is the first coordinate, $\mathbf{r}_2$ is the second
coordinate, and $\mathbf{r}_n$ is the coordinate at which the derivative is
evaluated.

Since $\gamma_{ij}$ is symmetric,

$$
\begin{equation}
  \left.
    \frac{\partial^{p_x + p_y + p_z}}{\partial x_1^{p_x} \partial y_1^{p_y} \partial z_1^{p_z}}
    \frac{\partial^{q_x + q_y + q_z}}{\partial x_2^{q_x} \partial y_2^{q_y} \partial z_2^{q_z}}
    \gamma(\mathbf{r}_1, \mathbf{r}_2)
  \right|_{\mathbf{r}_1 = \mathbf{r}_2 = \mathbf{r}_n} =
  \left.
    \frac{\partial^{q_x + q_y + q_z}}{\partial x_1^{q_x} \partial y_1^{q_y} \partial z_1^{q_z}}
    \frac{\partial^{p_x + p_y + p_z}}{\partial x_2^{p_x} \partial y_2^{p_y} \partial z_2^{p_z}}
    \gamma(\mathbf{r}_1, \mathbf{r}_2)
  \right|_{\mathbf{r}_1 = \mathbf{r}_2 = \mathbf{r}_n}
\end{equation}
$$

### 1.4.1 Stress Tensor

$$
\begin{equation}
  \begin{split}
    \boldsymbol{\sigma}_{ij}(\mathbf{r}_n | \alpha, \beta)
    =&
    -\frac{1}{2} \alpha
    \left(
      \frac{\partial^2}{\partial r_i \partial r'_j} \gamma(\mathbf{r}, \mathbf{r}')
      + \frac{\partial^2}{\partial r_j \partial r'_i} \gamma(\mathbf{r}, \mathbf{r}')
    \right)_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
    & +\frac{1}{2} (1 - \alpha)
    \left(
      \frac{\partial^2}{\partial r_i \partial r_j} \gamma(\mathbf{r}, \mathbf{r})
      + \frac{\partial^2}{\partial r'_i \partial r'_j} \gamma(\mathbf{r}, \mathbf{r}')
    \right)_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
    & - \frac{1}{2} \delta_{ij} \beta \nabla^2 \rho(\mathbf{r}_n)\\
    =&
    - \alpha
    \left.
      \frac{\partial^2}{\partial r_i \partial r'_j} \gamma(\mathbf{r}, \mathbf{r}')
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}
    + (1 - \alpha)
    \left.
      \frac{\partial^2}{\partial r_i \partial r_j} \gamma(\mathbf{r}, \mathbf{r})
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}
    - \frac{1}{2} \delta_{ij} \beta \nabla^2 \rho(\mathbf{r}_n)\\
  \end{split}
\end{equation}
$$

In [9]:
from gbasis.evals.stress_tensor import evaluate_stress_tensor

# evaluate stress tensor (α = 1 and β = 0) at one point using density matrix expressed w.r.t. atomic orbitals
stress_tensor_1_0_atomic = evaluate_stress_tensor(one_dm, basis, one_point)

# evaluate stress tensor (α = 0.5 and β = 1) at one point using density matrix expressed w.r.t. atomic orbitals
stress_tensor_point5_1_atomic = evaluate_stress_tensor(one_dm, basis, one_point, alpha=0.5, beta=1)

# evaluate stress tensor (α = 1 and β = 0) at one point using density matrix expressed w.r.t. molecular orbitals
stress_tensor_1_0_molecular = evaluate_stress_tensor(one_dm, basis, one_point, transform=transform)

print("Stress Tensor (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(stress_tensor_1_0_atomic)
print()
print("Stress Tensor (α = 0.5 and β = 1) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(stress_tensor_point5_1_atomic)
print()
print("Stress Tensor (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(stress_tensor_1_0_molecular)

Stress Tensor (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[[[-0.77717472  1.60935069  1.40229567]
  [ 1.60935069 -3.22194076 -2.79949973]
  [ 1.40229567 -2.79949973 -2.29947534]]]

Stress Tensor (α = 0.5 and β = 1) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[[[-12.31985162  -0.262827    -0.67665962]
  [ -0.262827    -8.65720855   3.01933972]
  [ -0.67665962   3.01933972  -9.66985457]]]

Stress Tensor (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[[[ -43.48271614  102.10737581   89.00543618]
  [ 102.10737581 -235.97175535 -205.83996535]
  [  89.00543618 -205.83996535 -179.56837671]]]


### 1.4.2 Ehrenfest Force

Ehrenfest force is defined as the negative of the divergence of the stress tensor:

$$
\begin{equation}
  \hspace{-4em}
  \begin{split}
    F_{j}(\mathbf{r}_n | \alpha, \beta)
    =&
    - \sum_i \frac{\partial}{\partial r_i} \boldsymbol{\sigma}_{ij}\\
    =&
    \alpha
    \sum_i
    \left.
      \frac{\partial^3}{\partial r^2_i \partial r'_j} \gamma(\mathbf{r}, \mathbf{r}')
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}
    + \alpha
    \sum_i
    \left.
      \frac{\partial^3}{\partial r_i \partial r'_i \partial r'_j} \gamma(\mathbf{r}, \mathbf{r}')
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
    &- (1 - \alpha)
    \sum_i
    \left.
      \frac{\partial^3}{\partial r^2_i \partial r_j} \gamma(\mathbf{r}, \mathbf{r})
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}
    - (1 - \alpha)
    \sum_i
    \left.
      \frac{\partial^3}{\partial r_i \partial r_j \partial r'_i} \gamma(\mathbf{r}, \mathbf{r})
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}
    + \frac{1}{2} \sum_i \delta_{ij} \beta
    \frac{\partial}{\partial r_i} \nabla^2 \rho(\mathbf{r}_n)\\
    =&
    \alpha
    \sum_i
    \left.
      \frac{\partial^3}{\partial r^2_i \partial r'_j} \gamma(\mathbf{r}, \mathbf{r}')
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
    &- (1 - \alpha)
    \sum_i
    \left.
      \frac{\partial^3}{\partial r^2_i \partial r_j} \gamma(\mathbf{r}, \mathbf{r})
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}
    - (1 - 2\alpha)
    \sum_i
    \left.
      \frac{\partial^3}{\partial r_i \partial r_j \partial r'_i} \gamma(\mathbf{r}, \mathbf{r})
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}
    + \frac{1}{2} \sum_i \delta_{ij} \beta
    \frac{\partial}{\partial r_i} \nabla^2 \rho(\mathbf{r}_n)\\
  \end{split}
\end{equation}
$$

In [10]:
from gbasis.evals.stress_tensor import evaluate_ehrenfest_force

# evaluate Ehrenfest force (α = 1 and β = 0) at one point using density matrix expressed w.r.t. atomic orbitals
ehren_force_1_0_atomic = evaluate_ehrenfest_force(one_dm, basis, one_point)

# evaluate Ehrenfest force (α = 0.5 and β = 1) at one point using density matrix expressed w.r.t. atomic orbitals
ehren_force_point5_1_atomic = evaluate_ehrenfest_force(one_dm, basis, one_point, alpha=0.5, beta=1)

# evaluate Ehrenfest force (α = 1 and β = 0) at one point using density matrix expressed w.r.t. molecular orbitals
ehren_force_1_0_molecular = evaluate_ehrenfest_force(one_dm, basis, one_point, transform=transform)

print("Ehrenfest Force (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(ehren_force_1_0_atomic)
print()
print("Ehrenfest Force (α = 0.5 and β = 1) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(ehren_force_point5_1_atomic)
print()
print("Ehrenfest Force (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(ehren_force_1_0_molecular)

Ehrenfest Force (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[[  8.31233434 -25.46254211 -21.90063464]]

Ehrenfest Force (α = 0.5 and β = 1) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[[ 16.94848775 -53.22056907 -44.69515864]]

Ehrenfest Force (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[[  518.52142456 -1898.95729258 -1693.92650237]]


### 1.4.3 Ehrenfest Hessian

$$
\begin{equation}
  \begin{split}
    H_{jk}(\mathbf{r}_n | \alpha, \beta)
    =&
    - \frac{\partial}{\partial r_k} F_j(\mathbf{r}_n | \alpha, \beta)\\
    =&
    \alpha
    \sum_i
    \left(
      \frac{\partial^4}{\partial r^2_i \partial r_k \partial r'_j} \gamma(\mathbf{r}, \mathbf{r}')
      +\frac{\partial^4}{\partial r^2_i \partial r'_j \partial r'_k} \gamma(\mathbf{r}, \mathbf{r}')
    \right)_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
    &- (1 - \alpha)
    \sum_i
    \left(
      \frac{\partial^4}{\partial r^2_i \partial r_j \partial r_k} \gamma(\mathbf{r}, \mathbf{r})
      + \frac{\partial^4}{\partial r^2_i \partial r_j \partial r'_k} \gamma(\mathbf{r}, \mathbf{r})
    \right)_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
    &- (1 - 2\alpha)
    \sum_i
    \left(
      \frac{\partial^4}{\partial r_i \partial r_j \partial r_k \partial r'_i} \gamma(\mathbf{r}, \mathbf{r})
      + \frac{\partial^4}{\partial r_i \partial r_j \partial r'_i \partial r'_k} \gamma(\mathbf{r}, \mathbf{r})
    \right)_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
    &+ \frac{1}{2} \sum_i \delta_{ij} \beta
    \frac{\partial^2}{\partial r_i \partial r_k} \nabla^2 \rho(\mathbf{r}_n)\\
  \end{split}
\end{equation}
$$

In [11]:
from gbasis.evals.stress_tensor import evaluate_ehrenfest_hessian

# evaluate Ehrenfest Hessian (α = 1 and β = 0) at one point using density matrix expressed w.r.t. atomic orbitals
ehren_hess_1_0_atomic = evaluate_ehrenfest_hessian(one_dm, basis, one_point)

# evaluate Ehrenfest Hessian (α = 0.5 and β = 1) at one point using density matrix expressed w.r.t. atomic orbitals
ehren_hess_point5_1_atomic = evaluate_ehrenfest_hessian(one_dm, basis, one_point, alpha=0.5, beta=1)

# evaluate Ehrenfest Hessian (α = 1 and β = 0) at one point using density matrix expressed w.r.t. molecular orbitals
ehren_hess_1_0_molecular = evaluate_ehrenfest_hessian(one_dm, basis, one_point, transform=transform)

print("Ehrenfest Hessian (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(ehren_hess_1_0_atomic)
print()
print("Ehrenfest Hessian (α = 0.5 and β = 1) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(ehren_hess_point5_1_atomic)
print()
print("Ehrenfest Hessian (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(ehren_hess_1_0_molecular)

Ehrenfest Hessian (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[[[-37.74953766 -49.78906788 -51.05138539]
  [-49.26668397 149.89202646 173.14530028]
  [-52.33192784 176.44169882 102.75623304]]]

Ehrenfest Hessian (α = 0.5 and β = 1) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[[[ -78.21389678  -98.50039769 -101.65964122]
  [ -97.97801378  275.53085505  326.70080588]
  [-102.94018367  329.99720442  179.57184668]]]

Ehrenfest Hessian (α = 1 and β = 0) Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[[[-2720.28324901 -3247.13838079 -3363.64035815]
  [-3244.59371098 10895.99557808 13105.07531223]
  [-3426.3938655  13257.63362655  8276.72940393]]]


### 1.4.4 Positive-Definite Kinetic Energy Density

$$
\begin{equation}
  \begin{split}
    t_+ (\mathbf{r}_n)
    &= \frac{1}{2} \left.
      \nabla_{\mathbf{r}} \cdot \nabla_{\mathbf{r}'} \gamma(\mathbf{r}, \mathbf{r}')
    \right|_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
    &= \frac{1}{2} \left(
      \frac{\partial^2}{\partial x \partial x'} \gamma(\mathbf{r}, \mathbf{r}')
      + \frac{\partial^2}{\partial y \partial y'} \gamma(\mathbf{r}, \mathbf{r}')
      + \frac{\partial^2}{\partial z \partial z'} \gamma(\mathbf{r}, \mathbf{r}')
    \right)_{\mathbf{r} = \mathbf{r}' = \mathbf{r}_n}\\
  \end{split}
\end{equation}
$$

In [12]:
from gbasis.evals.density import evaluate_posdef_kinetic_energy_density

# evaluate positive-definite kinetic energy density at one point using density matrix 
# expressed w.r.t. atomic orbitals
pos_def_T_atomic = evaluate_posdef_kinetic_energy_density(one_dm, basis, one_point)

# evaluate positive-definite kinetic energy density at one point using density matrix 
# expressed w.r.t. molecular orbitals
pos_def_T_molecular = evaluate_posdef_kinetic_energy_density(one_dm, basis, one_point, transform)

print("Positive-Definite Kinetic Energy Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(pos_def_T_atomic)
print()
print("Positive-Definite Kinetic Energy Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(pos_def_T_molecular)

Positive-Definite Kinetic Energy Density Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[3.14929541]

Positive-Definite Kinetic Energy Density Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[229.5114241]


### 1.4.5 General Form of Kinetic Energy Density

$$
\begin{equation}
  t_{\alpha} (\mathbf{r}_n) = t_+(\mathbf{r}_n) + \alpha \nabla^2 \rho(\mathbf{r}_n)
\end{equation}
$$

In [13]:
from gbasis.evals.density import evaluate_general_kinetic_energy_density

# evaluate general form of kinetic energy density (α = 1) at one point using density matrix
# expressed w.r.t. atomic orbitals
general_T_1_atomic = evaluate_general_kinetic_energy_density(one_dm, basis, one_point, 1)

# evaluate general form of kinetic energy density (α = 0.5) at one point using density matrix
# expressed w.r.t. atomic orbitals
general_T_point5_atomic = evaluate_general_kinetic_energy_density(one_dm, basis, one_point, 0.5)

# evaluate general form of kinetic energy density (α = 1) at one point using density matrix
# expressed w.r.t. molecular orbitals
general_T_1_molecular = evaluate_general_kinetic_energy_density(one_dm, basis, one_point, 1, transform)

print("General Form of Kinetic Energy Density (α = 1) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(general_T_1_atomic)
print()
print("General Form of Kinetic Energy Density (α = 0.5) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:")
print(general_T_point5_atomic)
print()
print("General Form of Kinetic Energy Density (α = 1) Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:")
print(general_T_1_molecular)

General Form of Kinetic Energy Density (α = 1) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[22.62795455]

General Form of Kinetic Energy Density (α = 0.5) Evaluated with Density Matrix Expressed w.r.t. Atomic Orbitals:
[12.88862498]

General Form of Kinetic Energy Density (α = 1) Evaluated with Density Matrix Expressed w.r.t. Molecular Orbitals:
[1655.59850929]
